<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Monte Carlo Tree Search on the Metr-LA dataset</h2>
</center>

---

In this notebook the explanations of the predicted events of the *STGNN* on the *Metr-LA* dataset are computed by means of an *Explainer*. The explainer is an MCTS algorithm that performs a localized search on the input graph of the instance to be explained. It returns the important subgraph which serves as the explanation of the predicted selected event outputed by the STGNN. The search space is cut by a transparent *global heuristic* that exploits traffic domain-specific characteristics and limits the search on a subgraph of the input graph composed of the top scoring nodes according to the global heuristic. The explainer is inspired by the paper *Explaining temporal graph models through an explorer-navigator framework* <a name="cite_paper"></a>[<sup>[1]</sup>](#note_paper).

Firstly, grid search is applied to the training set in order to tune the hyperparameters of the explainer, afterwards, the important subgraphs of each dataset are extracted and the results are evaluated.

For more detailed informations about the used functions, look into the corresponding docstrings inside the python files, inside the `src` folder.

---
<small>

<a name="note_paper"></a>[1]
W. Xia et al. “Explaining temporal graph models through an explorer-navigator framework”. In: *The
Eleventh International Conference on Learning Representations*, 2023.
URL: https://openreview.net/forum?id=BR_ZhvcYbGJ.
</small>

In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [4]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cpu"


# 1 Loading the Data
In this section the data is loaded

In [5]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'metr-la')

In [6]:
import pickle
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [7]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_metr_la.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the STGNN in evaluation mode.
spatial_temporal_gnn.eval();

C:\Users\Mrudula Madhavan\AppData\Local\Temp\ipykernel_20616\2043237369.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stgnn_checkpoints = torch.load(stgnn_checkpoints

In [8]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_metr_la.csv'),
    has_header=True)

In [9]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [10]:
import pickle

# Get the data scaler.
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [11]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train.npy'))
y_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train.npy'))
x_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train_time.npy'))
y_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train_time.npy'))

x_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val.npy'))
y_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val.npy'))
x_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val_time.npy'))
y_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val_time.npy'))

x_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test.npy'))
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test_time.npy'))

# 2 Explainer
In this section the explainer algorithm is applied through a grid search on the train dataset, then the results are evaluated.

The explainer firstly computes a global heuristic $c_{v_i, v_j}^t$ among all input nodes $v_i$ at timesteps $t$ and the nodes of the event to explain $v_j$ as:
$$ c_{v_i, v_j}^t = \frac{\Delta t}{\Delta d} + \frac{\exp (- (\Delta t + \Delta d))}{\Delta s} $$
Where $\Delta t$ is the temporal distance between $t$ and the last timestep of $v_j$, $\Delta d$ their spatial distance and $\Delta s$ the abdolute difference between the speed of $v_i$ at time $t$ and the average speed of $v_j$ in the event to explain.

Afterwards, it performs a MCTS using the input subgraph composed of top scoring nodes by the heuristic as the root of the tree. It performs a series of rollout that involve choosing a path from the root to a leaf node. At each branching, a node of the graph described in the tree node is removed and a child tree node is obtained. The leaf node respects a sparsity threshold, as its graph has a limited pre-defined number of nodes. In practice, given node of the tree $\mathcal{N}_i$:
    
1. A new tree node is expanded from $\mathcal{N}_i$ according to an action. The action consists in removing a specific traffic node in the graph described in $\mathcal{N}_i$.
2. A child node is selected from $\mathcal{N}_i$ among the expanded ones. In particular, it is selected the child node that maximizes exploration and exploitation.
3. If $\mathcal{N}_i$ is a leaf node, its reward is computed using the STGNN $\Phi$.
4. If $\mathcal{N}_i$ is a leaf node, its reward is backpropagated from it up to the path nodes until the root is reached to update information useful for exploration and exploitation in the tree search.

Ultimately, the final explanation outcome defined as the important subgraph is determined by the subgraph of the leaf node that attains the highest reward and which meets the specified sparsity threshold.

## 2.1 Spatial Distance Matrix
Firstly, a speed distance matrix is built in order to compute the shortest distance in kms for each pair of nodes. This matrix is built by firstly finding the shortest path between couple of nodes through the distance matrix $A$ and then computing the length of the path in kms.

Given a path $[n_1, ..., n_m]$, the distance among $n_1$ and $n_m$ is obtained as:
$$ \sum_{i}^{m -1} \text{geodesic}(n_i, n_{i+1})$$

In [12]:
from src.data.data_processing import get_distance_matrix

if not os.path.exists(
    os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy')):
    # Build the distance matrix between the nodes.
    distance_matrix = get_distance_matrix(
        adj_matrix,
        locations_df,
        node_pos_dict)

    # Save the distance matrix.
    np.save(os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'),
            distance_matrix)

else:
    # Load the distance matrix.
    distance_matrix = np.load(
        os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'))

## 2.2 Grid Search

Grid search is performed on a portion of the train dataset in order to find the best set of hyperparameters of the Explainer:
* `explanation size factor`: multiplied by the number of the nodes of the event to explain, defines the size of the important subgraph found by the explainer;
* `cut size factor`: multiplied by `explanation size factor`, defines the number of top scoring nodes of the input graph according to the global heuristic that have to be used to cut the search;
* `exploration weight`: parameter to weight the exploration in the MCTS;
* `n rollouts`: the number of simulations to perform in the MCTS.

The results are evaluated in terms of *Fidelity*$^-$ and *average time* to output the subgraph.

* **Fidelity**$^\textbf{-}$
    $$
        \text{Fidelity}^- = \frac{1}{N}  \sum_{i=1}^N (\Phi(\mathcal{G}_i) - \Phi({\mathcal{G}}_i^m))
    $$
    with $N$ the number of instances, $\Phi$ the STGNN, $\mathcal{G}_i$ the $i^th$ input graph and $\mathcal{G}_i^m$ the important subgraph.
    The error $\Phi(\mathcal{G}_i) - \Phi({\mathcal{G}}_i^m)$ is computed using *MAE*, *RMSE* and *MAPE*.

* **Average time:**
    It measures the average time employed to output the important subgraph by for each instance.

In [14]:
# from src.explanation.monte_carlo.explanation import apply_grid_search

# apply_grid_search(
#    x_train[::10],
#    y_train[::10],
#    distance_matrix,
#    spatial_temporal_gnn,
#    scaler,
 #   n_rollouts_list=[30, 50],
 #   explanation_size_factor_list=[2, 3],
#    cut_size_factor_list=[2],
#    exploration_weight_list=[5, 10, 20],
#    remove_value_list=[0.])

The best hyperparameters are expressed below.

In [15]:
CUT_SIZE_FACTOR = 2
EXPLANATION_SIZE_FACTOR = 2
EXPLORATION_WEIGHT = 20
N_ROLLOUTS = 50

## 2.3 Computing the explanations and Saving the Results
In this section the explanations of the predicted events of the train, validation and test datasets are computed using the selected hyperparameters.

The results are evaluated in terms of *Fidelity*$^-$ using *MAE*, *RMSE* and *MAPE*; *Average time*; *Fidelity*$^+$ using *MAE*, *RMSE* and *MAPE*; and *Sparsity*.

*Fidelity*$^-$ and *Average time* have already been introduced. For what concerns *Fidelity*$^+$ and *Sparsity*:

* **Fidelity**$^\textbf{+}$
    $$
        \text{Fidelity}^+ = \frac{1}{N}  \sum_{i=1}^N (\Phi(\mathcal{G}_i) - \Phi({\mathcal{G}}_i^{1 - m}))
    $$
    with $N$ the number of instances, $\Phi$ the STGNN, $\mathcal{G}_i$ the $i^th$ input graph and $\mathcal{G}_i^{1 - m}$ the complement of the important subgraph.
* **Sparsity:**
    $$\text{Sparsity} = \frac{1}{N} \sum_{i=1}^N (1 -\frac{m_i}{M_i})$$
    with $N$ the number of instances, $m_i$ the number of nodes in the important subgraph used for the explanation at instance $i$ and $M_i$ the number of nodes of the input graph at instance $i$.

In [16]:
import os
import numpy as np

EXPLAINED_DATA_DIR = os.path.join(BASE_DATA_DIR, 'explained')
os.makedirs(EXPLAINED_DATA_DIR, exist_ok=True)


In [ ]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the training set...')
x_train_explained, y_train_explained, train_scores = get_all_explanations(
    x_train,
    y_train,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train.npy'), x_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train.npy'), y_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'train_scores.npy'), train_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train_time.npy'), x_train_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train_time.npy'), y_train_time)


Computing the explanations for the training set...
[5/999] - 86s - MAE: { severe_congestion 2.9 -congestion N/D -free_flow N/D - total: 2.39 } - RMSE: { severe_congestion 4.01 -congestion N/D -free_flow N/D - total: 3.25 } - MAPE: { severe_congestion 10.5% -congestion N/D% -free_flow N/D% - total: 8.7% } - Average time: 17.3s                

In [ ]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the validation set...')
x_val_explained, y_val_explained, val_scores = get_all_explanations(
    x_val,
    y_val,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val.npy'), x_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val.npy'), y_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'val_scores.npy'), val_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val_time.npy'), x_val_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val_time.npy'), y_val_time)

In [ ]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the test set...')
x_test_explained, y_test_explained, test_scores = get_all_explanations(
    x_test,
    y_test,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test.npy'), x_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test.npy'), y_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'test_scores.npy'), test_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test_time.npy'), x_test_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test_time.npy'), y_test_time)

In [ ]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_train.npy'))

x_val_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_val.npy'))

x_test_explained = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test.npy'))

In [ ]:
from src.explanation.monte_carlo.explanation import print_all_fidelity_plus


print('Computing the fidelity+ for the training set...')
print_all_fidelity_plus(x_train, y_train, x_train_explained, spatial_temporal_gnn, scaler, remove_value=0.)
print()

print('Computing the fidelity+ for the validation set...')
print_all_fidelity_plus(x_val, y_val, x_val_explained, spatial_temporal_gnn, scaler, remove_value=0.)
print()

print('Computing the fidelity+ for the test set...')
print_all_fidelity_plus(x_test, y_test, x_test_explained, spatial_temporal_gnn, scaler, remove_value=0.)
print()

In [ ]:
from src.explanation.monte_carlo.explanation import print_all_sparsity


print('Computing the sparsity for the training set...')
print_all_sparsity(x_train, x_train_explained)
print()

print('Computing the sparsity for the validation set...')
print_all_sparsity(x_val, x_val_explained)
print()

print('Computing the sparsity for the test set...')
print_all_sparsity(x_test, x_test_explained)
print()